In [1]:
import json
import pandas as pd
import numpy as np
from emoji import core
import re
import ast
from matplotlib import pyplot as plt

In [225]:
playlists = pd.read_json('../0_data/playlists.json', typ = 'frame', encoding='utf-8')
song_meta = pd.read_json('../0_data/song_meta.json', typ = 'frame', encoding='utf-8') 
genre_gn_all = pd.read_json('../0_data/genre_gn_all.json', typ = 'frame', encoding='utf-8', orient='index')

### 1. song_meta

> 1. 날짜 오류 수정

In [36]:
# 날짜 오류있는 행 확인
song_meta[song_meta['issue_date'] > 20220000] # index : 141185, 448286

indices_to_update = [141185, 448286]
new_value = 20200113 # 알맞는 발매날짜
# 특정 행의 'issue_date' 값을 변경
for idx in indices_to_update:
    song_meta.at[idx, 'issue_date'] = new_value

In [37]:
# 알맞게 변경되었는지 확인
song_meta.iloc[[141185, 448286], :]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
141185,[GN1801],20200113,고요한 밤바다 구경하기 [여수 바다],10403230,[2737142],잔잔한 바람이 불어와 (There is a gentle breeze),[GN1800],[무드홀릭 (Moodholic)],141185
448286,[GN1801],20200113,고요한 밤바다 구경하기 [여수 바다],10403230,[2737142],여수 바다 (Yeosu sea),[GN1800],[무드홀릭 (Moodholic)],448286


> 2. 대분류 장르 GN9000노래 삭제

In [38]:
song_meta[song_meta['song_gn_gnr_basket'].apply(lambda x: 'GN9000' in x)] # 1834개
song_meta = song_meta[~song_meta['song_gn_gnr_basket'].apply(lambda x: 'GN9000' in x)] # 삭제

In [39]:
# 삭제되었는지 확인
song_meta[song_meta['song_gn_gnr_basket'].apply(lambda x: 'GN9000' in x)]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id


In [40]:
# song_meta 저장
song_meta.to_json('../0_data/song_meta_filter.json', orient='records')

### 2. playlists

> 1. 타이틀, 태그 불용어 처리

- 타이틀 특수문자 제거

In [226]:
# 이모지 제거 함수
def replace_emoji(inputString):
    return core.replace_emoji(inputString, replace='')

# 특수문자 제거 함수
def clean_text(inputString):
  text_rmv = re.sub('[-=+,#/\?:^●○.@*\"※~ㆍ!<>$♥❤♡☆★♪♬♩¶♨①๑｡＊②ఇ¥◈ ͡✔ ͡° ͜ʖ▶◀◎ღ✿ﾉ ;\\\_■『│』ᕷⓗ‘|\(\)\[\]`\'…》\”\“\’·]',' ', inputString)
  return text_rmv

In [227]:
# # 이모지 제거 함수 적용
playlists['plylst_title'] = playlists['plylst_title'].apply(replace_emoji)

# 특수문자 제거 함수 적용
playlists['plylst_title'] = playlists['plylst_title'].apply(clean_text)

In [228]:
# 양 옆 공백, %, & 제거

playlists['plylst_title'] = playlists['plylst_title'].str.strip()
playlists['plylst_title'] = playlists['plylst_title'].str.strip('%')
playlists['plylst_title'] = playlists['plylst_title'].str.strip('&')

In [229]:
playlists.iloc[[56134, 59162],:] # 확인

,tags,id,plylst_title,songs,like_cnt,updt_date
56134,[흥폭발],22525,내적댄스 터져베이베 아이돌 노래노래,"[108003, 594076, 297411, 97189, 35713, 445680,...",41,2018-06-15 19:24:26.000
59162,"[가을, 설렘, 사랑]",143039,가을이다 o 곧 겨울이다 T T,"[13142, 5049, 503816, 70166, 578529, 336829, 2...",1,2018-07-31 10:55:04.000


- 태그 특수문자 제거

In [230]:
# 특수문자 '_'가 있는 행 찾기
# '_'가 있는 행 찾기
rows_with_underscore = playlists['tags'][playlists['tags'].apply(lambda x: any('_' in tag for tag in x))]
rows_with_underscore

148                            [비오는날, 비_오는_날, 휴식, 비_오는날, 비]
285                                [상쾌해지는, 호불호없는, 모두가_흥얼흥얼]
364        [추천, 일렉트로니카, 5월_4주차, 주간, 최신, 전지음악, 인기, 주간일렉트로니카]
409                      [들었으면, 인디음악, 한번쯤, 이미지는_직접찍은사진, 인디]
522       [힙합, label_crew, 랩, 국내힙합, 트렌드, 쇼미더머니, 스웨그, aom...
                                ...                        
114625                                   [칠_Chill, 설레는_달콤함]
114647                     [우울, 위로, 위로가_필요할_때, 슬픔, 이별, 힘든날]
114688    [어쿠스틱, 혼자, 발라드, 우울, 분위기, 멜로디, 띵곡들, 조용한, 자기_전, 인디]
114703    [패션위크, 뉴욕, 도시, 갈라쇼음악, 트렌디, 팝, 편집샵, i_know_it, ...
115068               [담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]
Name: tags, Length: 2252, dtype: object

In [231]:
# 특수 문자를 제거하는 함수 정의
def remove_special_characters(tag_list):
    cleaned_tags = []
    for tag in tag_list:
        cleaned_tag = re.sub(r'[^\uAC00-\uD7A30-9a-zA-Z\s]', '', tag).replace(" ", "")
        cleaned_tags.append(cleaned_tag)
    return cleaned_tags

# 함수 적용
playlists['tags'] = playlists['tags'].apply(remove_special_characters)

In [232]:
playlists.iloc[[148, 285, 115068], :] # 확인

,tags,id,plylst_title,songs,like_cnt,updt_date
148,"[비오는날, 비오는날, 휴식, 비오는날, 비]",32533,비오는날 감성 자극 듣기 좋은 어쿠스틱 피아노 발라드,"[133091, 527764, 468136, 260894, 22845, 148876...",14,2020-02-16 00:56:45.000
285,"[상쾌해지는, 호불호없는, 모두가흥얼흥얼]",151357,기분좋아지는 노래들,"[270061, 158766, 573179, 272062, 37027, 245028...",0,2019-11-24 05:49:55.000
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의이야기, 사랑, 친구]",11343,1 눈물이 앞을 가리는 나의 이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000


In [233]:
# 플레이리스트 타이틀이 1글자인 조건 만들기
playlists_title_1str = playlists[playlists['plylst_title'].str.len() == 1]
playlists_title_1str = list(playlists_title_1str['id'].values) # 24 -> 롹, 잠, 쿨, 힙 등이며 대부분 좋아요 0 또는 1
# playlists[playlists['id'] == 98628]

# 삭제
playlists = playlists[playlists['plylst_title'].str.len() != 1]

In [234]:
# 빈 타이틀 삭제
playlists = playlists[playlists['plylst_title'] != '']

# 숫자로만 구성된 타이틀 삭제
numeric_title_rows = playlists[playlists['plylst_title'].str.isnumeric()]
indices_to_remove = numeric_title_rows.index
playlists = playlists.drop(indices_to_remove)

playlists.info()


# 115071 - 114707 = 364행 삭제

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114707 entries, 0 to 115070
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   tags          114707 non-null  object
 1   id            114707 non-null  int64 
 2   plylst_title  114707 non-null  object
 3   songs         114707 non-null  object
 4   like_cnt      114707 non-null  int64 
 5   updt_date     114707 non-null  object
dtypes: int64(2), object(4)
memory usage: 6.1+ MB


> 2. 타이틀 중복 제거(기준 : 노래 개수 or 좋아요 개수)

In [235]:
# 1. 플레이리스트 아이디(id)와 플레이리스트 명(plylst_title) 추출
plylst_title = playlists[['id', 'plylst_title']]

# 2. 플레이리스트 이름 별 플레이리스트 ID count 테이블 생성 : plylst_id_cnt
plylst_id_cnt = plylst_title.groupby('plylst_title').id.nunique().reset_index(name = 'mapping_plylst_cnt')

# 3. 플레이리스트 명 중복 구분 : 서로 다른 플레이리스트 아이디로 겹치는 플레이리스트 이름이 없으면 '중복 없음', 그 외 '중복 있음'
plylst_id_cnt = plylst_id_cnt.assign(
    mapping_plylst_cnt_category = pd.cut(plylst_id_cnt['mapping_plylst_cnt'], [0, 1, np.inf], labels = ['중복 없음', '중복 있음'])
)
plylst_id_cnt
# [0, 1, np.inf] : 0이상 1미만 값 = 중복없음, 1이상의 값 = 중복있음

# 4. 중복 구분 별 플레이리스트 수 count 테이블 생성 : plylst_id_cnt_division
plylst_id_cnt_division = pd.DataFrame(plylst_id_cnt.groupby('mapping_plylst_cnt_category').count())
plylst_id_cnt_division

,plylst_title,mapping_plylst_cnt
mapping_plylst_cnt_category,,
중복 없음,106525,106525
중복 있음,2722,2722


In [236]:
# 플레이리스트 타이틀을 기준으로 merge해 중복된 플레이리스트만 확인
playlists_dup = pd.merge(playlists, plylst_id_cnt, on='plylst_title')
playlists_dup= playlists_dup[playlists_dup['mapping_plylst_cnt_category'] == '중복 있음']
playlists_dup['song_cnt'] = playlists_dup['songs'].apply(len)
playlists_dup['tag_cnt'] = playlists_dup['tags'].apply(len)
playlists_dup.head()

,tags,id,plylst_title,songs,like_cnt,updt_date,mapping_plylst_cnt,mapping_plylst_cnt_category,song_cnt,tag_cnt
16,"[감성, 질리지않는, 나만알고싶은, Pop]",1516,나만 알고싶은 노래들,"[331055, 99287, 310974, 376435, 146989, 430106...",1,2018-01-02 12:10:59.000,4,중복 있음,31,4
17,"[기분전환, 까페]",86179,나만 알고싶은 노래들,"[480142, 578916, 467225, 701072, 197046, 63557...",6,2016-10-06 02:08:40.000,4,중복 있음,46,2
18,"[숨은명곡, 좋은노래]",54673,나만 알고싶은 노래들,"[113618, 422482, 380069, 75425, 79141, 657137,...",7,2016-07-15 10:20:17.000,4,중복 있음,23,2
19,"[감성힙합, 나만알고싶은, 뿜뿜, 그루브, 노래, 감성]",135616,나만 알고싶은 노래들,"[470548, 603591, 382764, 288501, 187291, 37802...",30,2018-07-13 08:18:54.000,4,중복 있음,45,6
44,"[기분전환, 가을]",105328,트렌디한 Cafe Pop Music,"[419702, 403579, 472776, 502078, 662300, 44363...",55,2014-09-18 11:29:55.000,2,중복 있음,16,2


In [237]:
# 같은 타이틀을 가진 플레이리스트의 노래 개수 비교
# playlists_dup에서 타이틀별로 groupby 후 song_cnt 값들을 song_cnt_nunique(새 컬럼)에 리스트 형태로 넣은 후 중복을 없애기 위해 set 진행
# 1의 의미는 노래 갯수가 중복되어 있다는 뜻이기 때문에 1의 갯수가 적은 지표를 기준으로 중복 플레이리스트 제거를 진행해야함
from collections import Counter

playlists_dup_song_cnt = playlists_dup.groupby('plylst_title')['song_cnt'].apply(list).reset_index()
playlists_dup_song_cnt['song_cnt_nunique'] = playlists_dup_song_cnt['song_cnt'].apply(lambda x: len(set(x)))
Counter(playlists_dup_song_cnt['song_cnt_nunique'])

Counter({2: 1543,
         1: 510,
         3: 343,
         4: 130,
         5: 56,
         6: 47,
         7: 30,
         9: 9,
         14: 5,
         20: 2,
         8: 17,
         15: 5,
         11: 8,
         10: 5,
         16: 3,
         13: 3,
         25: 1,
         12: 2,
         19: 1,
         22: 1,
         17: 1})

In [238]:
# 같은 타이틀을 가진 플레이리스트의 태그 개수 비교

playlists_dup_tag_cnt = playlists_dup.groupby('plylst_title')['tag_cnt'].apply(list).reset_index()
playlists_dup_tag_cnt['tag_cnt_nunique'] = playlists_dup_tag_cnt['tag_cnt'].apply(lambda x: len(set(x)))
Counter(playlists_dup_tag_cnt['tag_cnt_nunique'])

Counter({1: 1225, 3: 183, 2: 1222, 4: 59, 5: 20, 6: 9, 7: 4})

In [239]:
# 같은 타이틀을 가진 플레이리스트의 좋아요 비교

playlists_dup_like_cnt = playlists_dup.groupby('plylst_title')['like_cnt'].apply(list).reset_index()
playlists_dup_like_cnt['like_cnt_nunique'] = playlists_dup_like_cnt['like_cnt'].apply(lambda x: len(set(x)))
Counter(playlists_dup_like_cnt['like_cnt_nunique'])

Counter({1: 431,
         2: 1741,
         3: 311,
         4: 102,
         5: 54,
         6: 32,
         7: 14,
         11: 4,
         9: 7,
         13: 2,
         10: 4,
         12: 5,
         8: 10,
         15: 1,
         14: 2,
         18: 1,
         24: 1})

- 플레이리스트 타이틀이 중복인 플레이리스트만 따로 모으고
- 그 중복 플리모음 중 노래갯수가 가장 많은 하나만 남겨두고
- 원래 train의 중복 없음만 따로 모아 중복모음과 concat

In [240]:
# train에 중복 갯수, 중복 여부 컬럼 추가, 플레이리스트 내 곡 갯수 추가
playlists_add_col = pd.merge(playlists, plylst_id_cnt, on='plylst_title')
playlists_add_col['song_cnt'] = playlists_add_col['songs'].apply(lambda x : len(x))

# 매핑갯수, 타이틀, 플리 내 노래 갯수, like_cnt 내림차순 정렬
playlists_filter = playlists_add_col.sort_values(['mapping_plylst_cnt', 'plylst_title', 'song_cnt', 'like_cnt'], ascending=False)
playlists_filter

,tags,id,plylst_title,songs,like_cnt,updt_date,mapping_plylst_cnt,mapping_plylst_cnt_category,song_cnt
8532,"[집중, 어쿠스틱, 혼자, 발라드, 우울, 드라이브, 산책, 조용조용, 띵곡들, 인디]",152785,혼자만의 시간이 필요할때,"[208812, 629738, 232874, 13281, 193232, 389468...",1,2020-03-26 06:14:54.000,43,중복 있음,81
8519,"[어쿠스틱, 혼자, 발라드, 우울, 분위기, 드라이브, 띵곡만, 산책, 운전, 인디]",127180,혼자만의 시간이 필요할때,"[570836, 209135, 389920, 105140, 582252, 38261...",1,2020-03-24 06:39:20.000,43,중복 있음,78
8536,"[띵곡, 혼자만, 어쿠스틱, 발라드, 우울, 분위기, 드라이브, 조용한, 산책, 인디]",76448,혼자만의 시간이 필요할때,"[570836, 209135, 389920, 105140, 582252, 38261...",1,2020-04-13 05:56:55.000,43,중복 있음,78
8541,"[어쿠스틱, 혼자, 발라드, 우울, 분위기, 드라이브, 조용한, 산책, 띵곡들, 인디]",141114,혼자만의 시간이 필요할때,"[484964, 626369, 570836, 366786, 413837, 20913...",1,2020-04-22 06:38:57.000,43,중복 있음,78
8548,"[어쿠스틱, 혼자, 발라드, 분위기, 드라이브, 조용한, 산책, 운전, 띵곡들, 인디]",18201,혼자만의 시간이 필요할때,"[484964, 626369, 570836, 366786, 413837, 20913...",1,2020-04-06 05:20:46.000,43,중복 있음,78
...,...,...,...,...,...,...,...,...,...
31846,[일렉],56144,00 Party 일렉트로댄스,"[256418, 183424, 431998, 654442, 354592, 24132...",7,2018-02-06 16:02:42.000,1,중복 없음,20
17805,[일렉],58768,00 Electronica EDM,"[673003, 251403, 617690, 441958, 436128, 62788...",5,2020-01-05 00:24:22.000,1,중복 없음,133
108585,[일렉],110536,0 일렉듣고 광질준비 O K ElecVVIP,"[349306, 93890, 651902, 46845, 362347, 330768,...",173,2012-11-29 14:34:56.000,1,중복 없음,59
52783,"[비오는날, 추억, 회상]",106660,비오는날 듣기 좋은 연주곡 모음,"[79329, 51922, 173875, 216115, 422584, 509605,...",6,2015-10-06 11:45:24.000,1,중복 없음,25


In [241]:
playlists_dup = playlists_filter[playlists_filter['mapping_plylst_cnt_category'] == '중복 있음'] # 중복 플리 모음
playlists_dup['plylst_title'].nunique() # 중복된 플레이리스트의 유니크 타이틀 갯수 : 2723

2722

In [242]:
playlists_dup['tag_cnt'] = playlists_dup['tags'].apply(len)
playlists_dup.head()

C:\Users\cg063\AppData\Local\Temp\ipykernel_14084\2448186990.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playlists_dup['tag_cnt'] = playlists_dup['tags'].apply(len)


,tags,id,plylst_title,songs,like_cnt,updt_date,mapping_plylst_cnt,mapping_plylst_cnt_category,song_cnt,tag_cnt
8532,"[집중, 어쿠스틱, 혼자, 발라드, 우울, 드라이브, 산책, 조용조용, 띵곡들, 인디]",152785,혼자만의 시간이 필요할때,"[208812, 629738, 232874, 13281, 193232, 389468...",1,2020-03-26 06:14:54.000,43,중복 있음,81,10
8519,"[어쿠스틱, 혼자, 발라드, 우울, 분위기, 드라이브, 띵곡만, 산책, 운전, 인디]",127180,혼자만의 시간이 필요할때,"[570836, 209135, 389920, 105140, 582252, 38261...",1,2020-03-24 06:39:20.000,43,중복 있음,78,10
8536,"[띵곡, 혼자만, 어쿠스틱, 발라드, 우울, 분위기, 드라이브, 조용한, 산책, 인디]",76448,혼자만의 시간이 필요할때,"[570836, 209135, 389920, 105140, 582252, 38261...",1,2020-04-13 05:56:55.000,43,중복 있음,78,10
8541,"[어쿠스틱, 혼자, 발라드, 우울, 분위기, 드라이브, 조용한, 산책, 띵곡들, 인디]",141114,혼자만의 시간이 필요할때,"[484964, 626369, 570836, 366786, 413837, 20913...",1,2020-04-22 06:38:57.000,43,중복 있음,78,10
8548,"[어쿠스틱, 혼자, 발라드, 분위기, 드라이브, 조용한, 산책, 운전, 띵곡들, 인디]",18201,혼자만의 시간이 필요할때,"[484964, 626369, 570836, 366786, 413837, 20913...",1,2020-04-06 05:20:46.000,43,중복 있음,78,10


In [243]:
# playlists_dup 중 각 플리 제목 별 노래갯수가 가장 많은 하나만 남긴다
playlists_dup_filter = pd.DataFrame(playlists_dup[playlists_dup.groupby('plylst_title')['song_cnt'].transform(max) == playlists_dup['song_cnt']]) # 3572 
playlists_dup_filter = pd.DataFrame(playlists_dup_filter[playlists_dup_filter.groupby('plylst_title')['like_cnt'].transform(max) == playlists_dup_filter['like_cnt']]) # 2928 
playlists_dup_filter = pd.DataFrame(playlists_dup_filter[playlists_dup_filter.groupby('plylst_title')['tag_cnt'].transform(max) == playlists_dup_filter['tag_cnt']]) # 2909 
playlists_dup_filter = pd.DataFrame(playlists_dup_filter[playlists_dup_filter.groupby('plylst_title')['updt_date'].transform(max) == playlists_dup_filter['updt_date']]) # 2748 
playlists_dup_filter

,tags,id,plylst_title,songs,like_cnt,updt_date,mapping_plylst_cnt,mapping_plylst_cnt_category,song_cnt,tag_cnt
8532,"[집중, 어쿠스틱, 혼자, 발라드, 우울, 드라이브, 산책, 조용조용, 띵곡들, 인디]",152785,혼자만의 시간이 필요할때,"[208812, 629738, 232874, 13281, 193232, 389468...",1,2020-03-26 06:14:54.000,43,중복 있음,81,10
3119,[비오는날듣기좋은노래],75012,비오는날,"[65672, 522825, 145252, 146780, 500731, 473608...",22,2017-08-19 15:16:36.000,34,중복 있음,200,1
5536,"[조용히, 어쿠스틱, 여유, 발라드, 분위기, 힐링, 자기전이나, 띵곡, 인디, 편안한]",52312,자기전 하루한곡 발라드,"[570836, 209135, 389920, 105140, 582252, 38261...",1,2020-04-02 06:54:25.000,31,중복 있음,82,10
8429,[금요일],48129,금요일,"[614756, 147427, 340332, 675466, 414036, 61619...",0,2017-03-10 17:31:34.000,30,중복 있음,20,1
7288,[발라드],5960,추억,"[208821, 481749, 565402, 362197, 407236, 46604...",10,2010-10-21 09:45:12.000,27,중복 있음,85,1
...,...,...,...,...,...,...,...,...,...,...
155,"[기분좋은, 100번]",148495,100번 들어도 기분좋은 가요,"[208186, 155952, 669617, 438439, 397412, 60398...",4,2016-10-31 13:59:41.000,2,중복 있음,16,2
76640,[Pop],62278,100% 로맨틱 조지 마이클 223,"[366485, 435804, 442680, 538949, 263360, 29495...",0,2019-03-27 15:27:44.000,2,중복 있음,34,1
34444,[팝],50564,1 HITS OF DECADE SPECIAL 2,"[60025, 326802, 82991, 533287, 404318, 82686, ...",5,2009-07-11 21:37:42.000,2,중복 있음,37,1
75854,[팝],109558,1 HITS OF DECADE SPECIAL,"[289618, 78629, 349460, 218356, 189583, 613209...",5,2009-07-05 18:23:47.000,2,중복 있음,39,1


In [244]:
playlists_dup_filter[playlists_dup_filter['plylst_title'] == 'qqqa'] # 필터링 되지 않은 중복 타이틀은 플레이리스트 자체가 같음

,tags,id,plylst_title,songs,like_cnt,updt_date,mapping_plylst_cnt,mapping_plylst_cnt_category,song_cnt,tag_cnt
50442,[여름],7524,qqqa,"[522825, 455668, 549178, 407828, 343974]",1,2015-11-04 17:51:14.000,3,중복 있음,5,1
50443,[여름],43031,qqqa,"[522825, 455668, 549178, 407828, 343974]",1,2015-11-04 17:51:14.000,3,중복 있음,5,1


In [245]:
# 타이틀이 같은 플레이리스트 중 하나만 남긴다
playlists_dup_filter = pd.DataFrame(playlists_dup_filter.groupby('plylst_title', group_keys=False).apply(lambda x: x.loc[x['song_cnt'].idxmax()]))
playlists_dup_filter = playlists_dup_filter.reset_index(drop=True) # 2722

In [246]:
# playlists_dup_filter 중 중복 여부 확인

# 1. 플레이리스트 아이디(id)와 플레이리스트 명(plylst_title) 추출
plylst_title_2 = playlists_dup_filter[['id', 'plylst_title']]

# 2. 플레이리스트 이름 별 플레이리스트 ID count 테이블 생성 : plylst_id_cnt
plylst_id_cnt_2 = plylst_title_2.groupby('plylst_title').id.nunique().reset_index(name = 'mapping_plylst_cnt')

# 3. 플레이리스트 명 중복 구분 : 서로 다른 플레이리스트 아이디로 겹치는 플레이리스트 이름이 없으면 '중복 없음', 그 외 '중복 있음'
plylst_id_cnt_2 = plylst_id_cnt_2.assign(
    mapping_plylst_cnt_category = pd.cut(plylst_id_cnt_2['mapping_plylst_cnt'], [0, 1, np.inf], labels = ['중복 없음', '중복 있음'])
)
plylst_id_cnt_2
# [0, 1, np.inf] : 0이상 1미만 값 = 중복없음, 1이상의 값 = 중복있음

# 4. 중복 구분 별 플레이리스트 수 count 테이블 생성 : plylst_id_cnt_division
plylst_id_cnt_division_2 = pd.DataFrame(plylst_id_cnt_2.groupby('mapping_plylst_cnt_category').count())
plylst_id_cnt_division_2

,plylst_title,mapping_plylst_cnt
mapping_plylst_cnt_category,,
중복 없음,2722,2722
중복 있음,0,0


In [247]:
# 중복 없음 파일만 만들기
playlists_no_dup = pd.merge(playlists, plylst_id_cnt, on='plylst_title')
playlists_no_dup = playlists_no_dup[playlists_no_dup['mapping_plylst_cnt_category'] == '중복 없음']
playlists_no_dup # 106525행

,tags,id,plylst_title,songs,like_cnt,updt_date,mapping_plylst_cnt,mapping_plylst_cnt_category
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,1,중복 없음
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,1,중복 없음
2,"[까페, 잔잔한]",76951,편하게 잔잔하게 들을 수 있는 곡,"[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,1,중복 없음
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,1,중복 없음
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,1,중복 없음
...,...,...,...,...,...,...,...,...
114702,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E SM 2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000,1,중복 없음
114703,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000,1,중복 없음
114704,"[담시, 가족, 눈물, 그리움, 주인공, 나의이야기, 사랑, 친구]",11343,1 눈물이 앞을 가리는 나의 이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000,1,중복 없음
114705,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000,1,중복 없음


In [269]:
playlists_dup_proc = pd.concat([playlists_dup_filter, playlists_no_dup], axis=0)
playlists_dup_proc # 2722+106525 =109247

# 114707 - 109247 = 타이틀 중복 제거로 5460행 삭제

,tags,id,plylst_title,songs,like_cnt,updt_date,mapping_plylst_cnt,mapping_plylst_cnt_category,song_cnt,tag_cnt
0,[Pop],123815,1 1 환상의 케미스트리 211,"[243181, 520307, 291514, 496987, 285795, 68548...",0,2019-03-27 15:27:43.000,2,중복 있음,36.0,1.0
1,[팝],109558,1 HITS OF DECADE SPECIAL,"[289618, 78629, 349460, 218356, 189583, 613209...",5,2009-07-05 18:23:47.000,2,중복 있음,39.0,1.0
2,[팝],50564,1 HITS OF DECADE SPECIAL 2,"[60025, 326802, 82991, 533287, 404318, 82686, ...",5,2009-07-11 21:37:42.000,2,중복 있음,37.0,1.0
3,[Pop],62278,100% 로맨틱 조지 마이클 223,"[366485, 435804, 442680, 538949, 263360, 29495...",0,2019-03-27 15:27:44.000,2,중복 있음,34.0,1.0
4,"[기분좋은, 100번]",148495,100번 들어도 기분좋은 가요,"[208186, 155952, 669617, 438439, 397412, 60398...",4,2016-10-31 13:59:41.000,2,중복 있음,16.0,2.0
...,...,...,...,...,...,...,...,...,...,...
114702,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E SM 2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000,1,중복 없음,NaN,NaN
114703,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000,1,중복 없음,NaN,NaN
114704,"[담시, 가족, 눈물, 그리움, 주인공, 나의이야기, 사랑, 친구]",11343,1 눈물이 앞을 가리는 나의 이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000,1,중복 없음,NaN,NaN
114705,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000,1,중복 없음,NaN,NaN


> 3. 플레이리스트 별 노래 개수, 좋아요 개수 이상치 행 날리기

- 노래 개수 5~104개 가진 플레이리스트만 남기기

In [274]:
playlists_dup_proc = playlists_dup_proc.iloc[:, :6]
playlists_dup_proc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109247 entries, 0 to 114706
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   tags          109247 non-null  object
 1   id            109247 non-null  int64 
 2   plylst_title  109247 non-null  object
 3   songs         109247 non-null  object
 4   like_cnt      109247 non-null  int64 
 5   updt_date     109247 non-null  object
dtypes: int64(2), object(4)
memory usage: 5.8+ MB


In [340]:
playlists_dup_proc['songs_cnt'].describe()

count    109247.000000
mean         46.047370
std          44.289386
min           1.000000
25%          19.000000
50%          30.000000
75%          53.000000
max         200.000000
Name: songs_cnt, dtype: float64

In [314]:
Q1 = 19
Q3 = 53
IQR = Q3 - Q1

print(Q1-(1.5*IQR))
print(Q3+(1.5*IQR))

-32.0
104.0


In [319]:
playlists_dup_proc['songs_cnt'] = playlists_dup_proc['songs'].apply(lambda x : len(x))
playlists_song_cut = playlists_dup_proc[(playlists_dup_proc['songs_cnt'] >= 5) & (playlists_dup_proc['songs_cnt'] < 105) ]  # 5~104개만 남기기
playlists_song_cut.info() # 109247 -98952 = 10295행 삭제

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98952 entries, 0 to 114705
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          98952 non-null  object
 1   id            98952 non-null  int64 
 2   plylst_title  98952 non-null  object
 3   songs         98952 non-null  object
 4   like_cnt      98952 non-null  int64 
 5   updt_date     98952 non-null  object
 6   songs_cnt     98952 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 6.0+ MB


- 좋아요 2개 이하(describe 시 25%가 2개임) 삭제

In [321]:
playlists_song_cut['like_cnt'].describe()

Q1 = 2
Q3 = 22
IQR = Q3 - Q1

print(Q1-(1.5*IQR))
print(Q3+(1.5*IQR))

-28.0
52.0


In [324]:
playlists_song_tag_cut = playlists_song_cut[playlists_song_cut['like_cnt'] > 2]
playlists_song_tag_cut = playlists_song_tag_cut.iloc[:, :6]
playlists_song_tag_cut.info()# 24860행 삭제

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74092 entries, 1 to 114705
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          74092 non-null  object
 1   id            74092 non-null  int64 
 2   plylst_title  74092 non-null  object
 3   songs         74092 non-null  object
 4   like_cnt      74092 non-null  int64 
 5   updt_date     74092 non-null  object
dtypes: int64(2), object(4)
memory usage: 4.0+ MB


> 4. song, 태그 빈도수에 따라 리스트에서 빼기 -> 리스트가 비워져있으면 행 날리기

- 1회 이하 사용된 tag 리스트에서 빼기

In [325]:
# tag usage count 계산
tag_counts = playlists_song_tag_cut.explode('tags')['tags'].value_counts(ascending=True)

# 1번 이하로 사용된 태그
unique_tags = list(tag_counts[tag_counts == 1].index)
unique_tags

['담시',
 '긴장완화',
 '여름아부탁해',
 '김현철',
 '최신댄스곡',
 '꿀아티스트',
 '묘하게',
 'nsc',
 '10센치',
 '기타치기좋은노래',
 '자전거탄풍경',
 '하타요가',
 '인도요가음악',
 '마음으로듣는노래',
 'ToSmith',
 'FromEliott',
 '어쿠스틱연주',
 '싱그러운봄',
 '쿨한노래',
 '멋진노래',
 '온더레코드',
 '아무장르',
 '상어',
 '라이언',
 '드림쇼',
 '에이스',
 '네오',
 'Bennington',
 '오자와세이지',
 'Chester',
 '프란시스풀랑크',
 'M83',
 '20s',
 '첫내한',
 '조슈아트리',
 '2019Tour',
 '이달의소녀오드아이써클',
 '들길',
 '마약같은일렉',
 '느낌있는일렉',
 '다른버전',
 '인디매력',
 '매력적인디',
 '두가지',
 '조하문',
 '마그마',
 '다채로운창법',
 '위로해준',
 '잘츠부르크페스티벌',
 '여름축제',
 '음악축제',
 '잘츠부르크',
 '바래',
 '카페에서듣는피아노연주곡',
 '남자여자보컬',
 'Justice',
 'bassmentjaxx',
 'ontherecord',
 '크리스토퍼내한',
 '로로스',
 '체스터',
 '에이제이',
 'abs',
 '지역',
 '오랜만에듣는명곡',
 '사랑과',
 '월요일월',
 '상쾌지수',
 '아무밴드',
 '레이니썬',
 '오딘',
 'MayuWakisaka',
 '마유와키사카',
 '빈지노팬',
 '다시들어도좋아',
 '잠이안오는밤',
 '카페에어울리는',
 '이온음료',
 '창작음악',
 '국악밴드',
 '마타도르',
 '댄스핫트랙',
 '1세대힙합',
 '1세대',
 '그리움을담은노래',
 '크리스마스연금',
 '겨울연금',
 '테크노808',
 '좌심방우심방',
 '소울담은플레이리스트',
 'YESEO',
 '바브라스트라이샌드',
 '뮤지션리그',
 '선호하는',
 'VLIVE',
 '신경꺼',
 '블랙히스토리먼스',
 '4월의뮤지

- tag_counts describe

In [359]:
tag_freq = pd.DataFrame(tag_counts).reset_index().rename(columns={'index':'tag_id', 'tags':'tag_cnt'})
tag_freq_group = tag_freq.groupby('tag_cnt').count().reset_index()
tag_freq_group['tag_cnt'].describe()

count      335.000000
mean       662.244776
std       1333.659647
min          1.000000
25%         84.500000
50%        188.000000
75%        439.000000
max      10528.000000
Name: tag_cnt, dtype: float64

In [360]:
Q1 = 84
Q3 = 439
IQR = Q3 - Q1

print(Q1-(1.5*IQR))
print(Q3+(1.5*IQR))

-448.5
971.5


In [326]:
# 1번만 사용된 태그들은 삭제
playlists_song_tag_cut['tags'] = playlists_song_tag_cut['tags'].apply(lambda x: [tag for tag in x if tag not in unique_tags])

In [327]:
# tags가 비어있지 않은 행만 남김
playlists_tag_cut = playlists_song_tag_cut[playlists_song_tag_cut['tags'].apply(lambda x: len(x) > 0)]
playlists_tag_cut.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
1,[팝],109558,1 HITS OF DECADE SPECIAL,"[289618, 78629, 349460, 218356, 189583, 613209...",5,2009-07-05 18:23:47.000
2,[팝],50564,1 HITS OF DECADE SPECIAL 2,"[60025, 326802, 82991, 533287, 404318, 82686, ...",5,2009-07-11 21:37:42.000
4,[기분좋은],148495,100번 들어도 기분좋은 가요,"[208186, 155952, 669617, 438439, 397412, 60398...",4,2016-10-31 13:59:41.000
8,"[겨울, 크리스마스]",117162,12월의 재즈,"[588425, 407630, 235568, 347651, 194467, 42313...",353,2016-12-26 12:33:33.000
9,[발라드],151638,12월의 크리스마스를 yeah,"[290351, 404382, 591246, 18938, 610105, 676958...",29,2009-12-22 09:11:45.000


In [328]:
playlists_tag_cut['tag_cnt'] = playlists_tag_cut['tags'].apply(len)
playlists_tag_cut['tag_cnt'].unique()

C:\Users\cg063\AppData\Local\Temp\ipykernel_14084\4293636261.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playlists_tag_cut['tag_cnt'] = playlists_tag_cut['tags'].apply(len)


array([ 1,  2,  3,  5,  4,  6, 10,  9,  8,  7, 11], dtype=int64)

In [329]:
playlists_tag_cut = playlists_tag_cut.iloc[:, :6]
playlists_tag_cut.info()

# 74092 - 73688 = 404행 삭제

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73688 entries, 1 to 114705
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          73688 non-null  object
 1   id            73688 non-null  int64 
 2   plylst_title  73688 non-null  object
 3   songs         73688 non-null  object
 4   like_cnt      73688 non-null  int64 
 5   updt_date     73688 non-null  object
dtypes: int64(2), object(4)
memory usage: 3.9+ MB


- 2회 이하 사용된 song 리스트에서 빼기

In [331]:
song_counts = playlists_tag_cut.explode('songs')['songs'].value_counts()
songs_to_remove = song_counts[song_counts <= 2].index
songs_to_remove

Int64Index([219250, 327278, 457985, 176461, 439867, 192016, 497692, 385090,
             54093, 493788,
            ...
            499026, 615635, 546701, 159437, 469852, 325339, 237246, 426610,
            132221,  74465],
           dtype='int64', length=301899)

- song_counts describe

In [356]:
song_freq = pd.DataFrame(song_counts).reset_index().rename(columns={'index':'song_id', 'songs':'song_cnt'})
song_freq_group = song_freq.groupby('song_cnt').count().reset_index()
song_freq_group['song_cnt'].describe()

count     541.000000
mean      298.304991
std       206.317917
min         1.000000
25%       136.000000
50%       271.000000
75%       423.000000
max      1151.000000
Name: song_cnt, dtype: float64

In [357]:
Q1 = 136
Q3 = 423
IQR = Q3 - Q1

print(Q1-(1.5*IQR))
print(Q3+(1.5*IQR))

-294.5
853.5


In [332]:
playlists_tag_cut['songs'] = playlists_tag_cut['songs'].apply(lambda songs: [song for song in songs if song not in songs_to_remove])

In [333]:
# 2회 이하 song이 없는지 확인
playlists_tag_cut.explode('songs')['songs'].value_counts()

205179    1151
144663    1147
116573    1125
366786    1025
357367     917
          ... 
626714       3
373567       3
536281       3
402134       3
702092       3
Name: songs, Length: 146930, dtype: int64

In [334]:
#songs가 비어있지 않은 행만 남김
playlists_tag_song_cut = playlists_tag_cut[playlists_tag_cut['songs'].apply(lambda x: len(x) > 0)]

# 비어있는 songs가 없는지 확인
playlists_tag_song_cut['song_cnt'] = playlists_tag_song_cut['songs'].apply(len)
playlists_tag_song_cut['song_cnt'].unique()

C:\Users\cg063\AppData\Local\Temp\ipykernel_14084\1251229739.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playlists_tag_song_cut['song_cnt'] = playlists_tag_song_cut['songs'].apply(len)


array([ 36,  32,  15,  21,  34,   9,  82,  22,  48,  18,  58,  27,  10,
         1,  14,  13,  30,  16,   2,   8,  17,  51,  29,  69,  23,  25,
        66,  24,   6,  70,  20,   5,  72,  19,  94,  68,  83,  97,  80,
        86,  43,  35,  84,  91,   4,  11,  33,  26,  40,  54,  28,  95,
        12,  52,  37,  31,  38,  39,  85,  64,  62,  74,  76,   7,  67,
         3,  61,  42,  46,  49,  87,  53,  60,  71,  92,  50,  79,  57,
        78,  41,  63, 103,  44,  77,  47,  99,  59,  88,  90,  55,  45,
        98,  65,  96,  81,  93,  75, 102,  73,  56,  89, 101, 104, 100],
      dtype=int64)

In [366]:
playlists_tag_song_cut = playlists_tag_song_cut.iloc[:, :6]
playlists_tag_song_cut.info()

# 73688  -73014  = 674행 제거

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73014 entries, 1 to 114705
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          73014 non-null  object
 1   id            73014 non-null  int64 
 2   plylst_title  73014 non-null  object
 3   songs         73014 non-null  object
 4   like_cnt      73014 non-null  int64 
 5   updt_date     73014 non-null  object
dtypes: int64(2), object(4)
memory usage: 5.9+ MB


In [367]:
playlists_tag_song_cut.explode('tags')['tags'].nunique() # 9266개
playlists_tag_song_cut.explode('songs')['songs'].nunique() # 146930개

146930

In [370]:
tag_counts2 = playlists_tag_song_cut.explode('tags')['tags'].value_counts()

In [371]:
tag_freq = pd.DataFrame(tag_counts2).reset_index().rename(columns={'index':'tag_id', 'tags':'tag_cnt'})
tag_freq_group = tag_freq.groupby('tag_cnt').count().reset_index()
tag_freq_group['tag_cnt'].describe()

count      334.000000
mean       658.431138
std       1331.752301
min          1.000000
25%         84.250000
50%        187.500000
75%        449.750000
max      10505.000000
Name: tag_cnt, dtype: float64

In [372]:
Q1 = 84
Q3 = 449
IQR = Q3 - Q1

print(Q1-(1.5*IQR))
print(Q3+(1.5*IQR))

-463.5
996.5


> 정리

=> song_meta : 날짜 수정, 장르 정보 없는 노래 제거  
=> playlists  
    - 타이틀 & 태그 불용어 삭제 : 특수문자, 빈 타이틀, 모두 숫자인 경우, 1글자인 경우 포함  
    - 타이틀 중복 제거 : 타이틀이 완벽하게 같을 경우 '1_노래 갯수 많은 순서 -> 2_좋아요 갯수 많은 순서 -> 3_태그 갯수 많은 순서 -> 4_최신 업데이트' 기준으로 1개씩 골라냄 (4가지를 했는데도 같은건 완벽히 같았기에 하나만 남김)  
    - 중복 제거 후 song_cnt describe 해 5~104개 남기기, like_cnt describe 3개 이상인 플레이리스트   
    - 1번 사용된 tag, 2번 이하 사용된song 제거  
  
  
=> 115071(원래 playlist) -73014(전처리 후) = 42057행 삭제

> 데이터 저장

- 최종 전처리 파일은 ../0_data/playlists_filter.json 으로 저장

In [368]:
# playlists 저장
playlists_tag_song_cut.to_json('../0_data/playlists_filter.json', orient='records')

---